In [1]:
import utils

In [5]:
import pandas as pd
import jinja2

from collections import OrderedDict
from json import dumps
from ipywidgets import interact
from IPython.html import widgets
from IPython.display import display, display_pretty, Javascript, HTML
from IPython.utils.traitlets import Any, Bool, Dict, List, Unicode
from threading import Lock
from urllib2 import urlopen


In [9]:
data = [('california', 1000),  ('nevada', 200), ('new york', 558)]

In [2]:
graph = utils.pickle_from_file('../../data/yago_hierarchy.pickle')

In [148]:
sorted_nodes = sorted(graph.nodes())
nodes = [{
            'name': str(x).replace('wordnet_', '').split('_')[:-1],
            'root': len(graph.predecessors(x))
    } for x in sorted_nodes]
links = [{
        'source': sorted_nodes.index(source),
        'target': sorted_nodes.index(target)
    } for source, target in graph.edges()]

In [149]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    }
});

<IPython.core.display.Javascript object>

In [153]:
d3_template = jinja2.Template(
    """
    // Based on http://bl.ocks.org/mbostock/3885304

    require(["d3"], function(d3) {
        var graph = {
            'nodes': {{ nodes }},
            'links': {{ links }}
        };
        
        var zoom = d3.behavior.zoom()
            .scaleExtent([-10, 10])
            .on("zoom", zoomed);

        d3.select("#chart_d3 svg").remove();

        var width = 960,
            height = 500;

        var force = d3.layout.force()
            .size([width, height])
            .charge(-150)
            .linkDistance(70)
            .on("tick", tick);

        var drag = force.drag()
            .on("dragstart", dragstart);

        var svg = d3.select("#chart_d3").append("svg")
            .attr("width", width)
            .attr("height", height)
            .call(zoom);
        
        var container = svg.append("g");
        
        // build the arrow.
        container.append("svg:defs").selectAll("marker")
            .data(["end"])      // Different link/path types can be defined here
          .enter().append("svg:marker")    // This section adds in the arrows
            .attr("id", String)
            .attr("viewBox", "0 -5 10 10")
            .attr("refX", 15)
            .attr("refY", -1.5)
            .attr("markerWidth", 3)
            .attr("markerHeight", 3)
            .attr("orient", "auto")
          .append("svg:path")
            .attr("d", "M0,-5L10,0L0,5");

        var link = container.selectAll(".link"),
            node = container.selectAll(".node");

        force.nodes(graph.nodes)
          .links(graph.links)
          .start();

        link = link.data(graph.links)
          .enter().append("line")
            .attr("class", "link")
            .attr("marker-end", "url(#end)");;

        node = node.data(graph.nodes)
            .enter().append("g")
              .attr("class", "node")
              .classed("root-node", function(d) {return d.root == 0})
              .call(force.drag);
        
        node.append("circle")
              .attr("class", "node")
              .attr("r", 4)
              .on("dblclick", dblclick)
        
        node.append("text")
          .attr("dx", 12)
          .attr("dy", ".18em")
          .text(function(d) { return d.name });

        function tick() {
          link.attr("x1", function(d) { return d.source.x; })
              .attr("y1", function(d) { return d.source.y; })
              .attr("x2", function(d) { return d.target.x; })
              .attr("y2", function(d) { return d.target.y; });

          node.attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; });
        }

        function dblclick(d) {
          d3.select(this).classed("fixed", d.fixed = false);
        }

        function dragstart(d) {
          d3.select(this).classed("fixed", d.fixed = true);
        }
        
        function zoomed() {
          container.attr("transform", "translate(" + d3.event.translate + ")scale(" + d3.event.scale + ")");
        }
    });
    """
)


In [154]:
display(HTML("""
    <style>
        .link {
          stroke: #ccc;
          stroke-width: 1.5px;
        }

        .node {
          cursor: move;
          fill: #000;
        }
        
        .node.root-node {
          fill: blue;
        }

        .node.fixed {
          fill: #f00;
        }
    </style>
    <div id="chart_d3"/>
"""))

In [155]:
display(Javascript(d3_template.render(nodes=nodes, links=links)))

<IPython.core.display.Javascript object>